In [1]:
import os
import re
import json
import glob

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from data_process import data_processor
from chunking import chunking
from prompt import prompt

/opt/anaconda3/envs/dmbi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
path = os.getcwd()
work_dir = os.path.join(path, 'work_dir')
os.makedirs(work_dir, exist_ok=True)


In [3]:
outcome_filename = 'outcome_definition.json'    # definition of the outcome terminologies. 
with open(os.path.join(path, outcome_filename)) as outcome_file:
    outcome_definition = json.load(outcome_file)

In [4]:
def data_processing(path, raw_data_folder):

    processor = data_processor(path=path)   # claim the processing object

    loader = processor._extract_text_from_pdf       
    papers = processor.load_all_papers(folder=raw_data_folder, loader=loader)        # load papers in from pdf
    sections = processor.extract_section(papers)              # automated cleaning.
    processor.save_to_text(sections, os.path.join(path, 'cleaned_papers'))       # save cleaning results as text file. 
    return sections

In [ ]:
# preprocess papers

paper = data_processing(path=work_dir, raw_data_folder='pdfs')

In [ ]:
# ''' raw factor evaluation
# input = a json file of sample paragraphs (contexts).
# chunks = each line in the input json file '''
# evaluation_file = os.path.join(work_dir, 'evaluation_dataset_2_1205.json')

# try:
#     with open(evaluation_file, "r") as file:
#         data = json.load(file)
# except FileNotFoundError:
#     print(f"Error: {evaluation_file} not found.")
# except json.JSONDecodeError:
#     print(f"Error: Could not decode JSON from {evaluation_file}. Check file format.")

# chunks = data['paragraphs']

In [ ]:
# cleaned_data_folder = os.path.join(work_dir, 'cleaned_papers/tochunk/')
# assert(os.path.exists(cleaned_data_folder))
# # Load data
# loader = DirectoryLoader(path=cleaned_data_folder, glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
# documents = loader.load()

os.path.splitext(os.path.basename(documents[0].metadata['source']))[0].split()[0]   

'75'

In [8]:
'''generate raw factors'''

''' chunking for papers'''

# Load data
# Define data directory
cleaned_data_folder = os.path.join(work_dir, 'cleaned_papers/tochunk/')
assert(os.path.exists(cleaned_data_folder))
# Load data
loader = DirectoryLoader(path=cleaned_data_folder, glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
documents = loader.load()

# Define chunking write-to directory
chunk_folder = os.path.join(work_dir, 'chunks')
os.makedirs(chunk_folder, exist_ok=True)
# Chunk
chunk = chunking(chunk_folder, chunk_size=5000)
chunks = chunk.make_chunks(documents)
# chunk.save_chunks_to_text(docs=chunks)
# chunk.save_chunk_for_later_loading(docs=chunks)

# response_dir = os.path.join(work_dir, 'responses/response.txt')
for i in range(len(documents)):
    # print(documents[i])
    # break
    target_outcome = outcome_definition[0]
    
    chunks = chunk.make_chunks([documents[i]])
    doc_index = os.path.splitext(os.path.basename(documents[i].metadata['source']))[0].split()[0]  
    print(f'Collecting factors in paper {doc_index}') 
    response_dir = os.path.join(work_dir, f'responses/response_{doc_index}.txt')

    generate_prompt = prompt(chunked_docs = chunks, outcome_definition = outcome_definition)
    message_func = generate_prompt._message_find_factors_to_result
    results = generate_prompt.generate_response(message_func, include_chunk=False)
    generate_prompt.write_summaries_to_txt(results, response_dir)

    

100%|██████████| 2/2 [00:00<00:00, 1368.01it/s]


processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3
processing chunk 4
processing chunk 5
processing chunk 6
processing chunk 7
Analyze results saved to text file. 
processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3
processing chunk 4
processing chunk 5
processing chunk 6
processing chunk 7
processing chunk 8
processing chunk 9
processing chunk 10
processing chunk 11
processing chunk 12
Analyze results saved to text file. 


### remove no-related-content responses

In [9]:
# remove no-related-content responses
def remove_paragraphs_with_string(input_file, output_file, target_string):
    with open(input_file, 'r') as file:
        content = file.read()
    
    # Split the content into paragraphs
    paragraphs = content.split('\n\n')
    
    # Filter out paragraphs containing the target string
    filtered_paragraphs = [
        paragraph for paragraph in paragraphs if target_string not in paragraph
    ]
    
    # Join the remaining paragraphs with double newline characters
    result_content = '\n\n'.join(filtered_paragraphs)
    
    # Write the result to the output file
    with open(output_file, 'w') as file:
        file.write(result_content)

In [10]:
response_folder = 'work_dir/raw_factors/'

# Using glob to find all .txt files in the directory and subdirectories
input_filenames = glob.glob(os.path.join(response_folder, '**', '*.txt'), recursive=True)
output_filenames = [input_filename[:-4]+'_clean.txt' for input_filename in input_filenames]
target_string = 'There are no related factors in this chunk'

for i in range(len(input_filenames)):
    print(f'Start cleaning {input_filenames[i]}')
    remove_paragraphs_with_string(input_file=input_filenames[i], output_file=output_filenames[i], target_string=target_string)
    print(f'Completed cleaning {input_filenames[i]}')

Start cleaning work_dir/raw_factors/response_199.txt
Completed cleaning work_dir/raw_factors/response_199.txt
Start cleaning work_dir/raw_factors/response_75.txt
Completed cleaning work_dir/raw_factors/response_75.txt
Start cleaning work_dir/raw_factors/response_49.txt
Completed cleaning work_dir/raw_factors/response_49.txt
Start cleaning work_dir/raw_factors/response_151.txt
Completed cleaning work_dir/raw_factors/response_151.txt
Start cleaning work_dir/raw_factors/response_623.txt
Completed cleaning work_dir/raw_factors/response_623.txt
Start cleaning work_dir/raw_factors/response_144.txt
Completed cleaning work_dir/raw_factors/response_144.txt
Start cleaning work_dir/raw_factors/response_642.txt
Completed cleaning work_dir/raw_factors/response_642.txt
Start cleaning work_dir/raw_factors/response_118.txt
Completed cleaning work_dir/raw_factors/response_118.txt


### count number of factors related to each outcome respectively

In [8]:
''' count number of factors related to each outcome respectively'''
import re
response_folder = 'work_dir/raw_factors/'

def count_factors(file_path):
    # Initialize counters for each category
    adherence_count = 0
    dropout_count = 0
    accessibility_count = 0

    # Open and read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize a variable to keep track of which block we are in
    current_block = None

    # Iterate through each line in the file
    for line in lines:
        # Strip whitespace from the line
        line = line.strip()

        # Check the start of a new factor block
        if line.startswith("The factors contributing to"):
            if "Adherence" in line:
                current_block = 'adherence'
            elif "Dropout" in line:
                current_block = 'dropout'
            elif "Accessibility" in line:
                current_block = 'accessibility'
            else:
                current_block = None  # Reset if it's an unexpected line format
                
        # Count factors based on the current block
        if current_block:
            # Count the factors that start with a dash
            if line.startswith("-"):
                if current_block == 'adherence':
                    adherence_count += 1
                elif current_block == 'dropout':
                    dropout_count += 1
                elif current_block == 'accessibility':
                    accessibility_count += 1
                    
            # Check for inline factors immediately after the category using quotes
            quoted_factors = re.findall(r'"(.*?)"', line)
            if quoted_factors:
                if current_block == 'adherence':
                    adherence_count += len(quoted_factors)
                elif current_block == 'dropout':
                    dropout_count += len(quoted_factors)
                elif current_block == 'accessibility':
                    accessibility_count += len(quoted_factors)
                continue  # Skip the rest of the processing for this line

            # Check for inline factors immediately after the category without quotes
            inline_factors = re.search(r'are:\s*(\S[^.]+)', line)
            if inline_factors:
                if current_block == 'adherence':
                    adherence_count += 1
                elif current_block == 'dropout':
                    dropout_count += 1
                elif current_block == 'accessibility':
                    accessibility_count += 1
                continue  # Skip the rest of the processing for this line

        

    # Return the counts as a tuple
    return adherence_count, dropout_count, accessibility_count

# Example usage (assuming 'input.txt' is your file name):
file_path = os.path.join(response_folder, '2nd round_evaluate_response_study 75-151_clean.txt')
adherence, dropout, accessibility = count_factors(file_path)
print(f"Adherence factors: {adherence}")
print(f"Dropout factors: {dropout}")
print(f"Accessibility factors: {accessibility}")


FileNotFoundError: [Errno 2] No such file or directory: 'work_dir/raw_factors/2nd round_evaluate_response_study 75-151_clean.txt'

### load and chunk for validation dataset

In [20]:
''' extract factors '''
''' chunking for papers'''

# Load data
# Define data directory
cleaned_data_folder = os.path.join(work_dir, 'factors/')
assert(os.path.exists(cleaned_data_folder))
# Load data
loader = DirectoryLoader(path=cleaned_data_folder, glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
documents = loader.load()

# Define chunking write-to directory
chunk_folder = os.path.join(work_dir, 'chunks')
os.makedirs(chunk_folder, exist_ok=True)
# Chunk
chunk = chunking(chunk_folder,chunk_size=5000, chunk_overlap=100)
chunks = chunk.make_chunks(documents)
# chunk.save_chunks_to_text(docs=chunks)
# chunk.save_chunk_for_later_loading(docs=chunks)


100%|██████████| 3/3 [00:00<00:00, 1943.01it/s]


In [21]:
chunks

[Document(metadata={'source': '/Users/yushuhuang/Documents/research/dmbi_proj/work_dir/factors/accessibility.txt'}, page_content='"Build on favourite tracks" (Atreya et al., 2018)\n"Provide longer and shorter track options" (Atreya et al., 2018)\n"Select male or female voices" (Atreya et al., 2018)\n"Flexible and accessible interventions" (Lange, 2020)\n"Flexible and accessible intervention" (Llaneza et al., 2022)\n"Allow personalised individual schedule" (Llaneza et al., 2022)\n"Live support options" (Llaneza et al., 2022)\n"Flexible and accessible interventions" (Lunsky et al., 2021) - The flexibility and accessibility of the interventions make them easier to reach and utilize, enhancing accessibility.\n"More instructions to participate in an online forum" (Lunsky et al., 2021) - Providing more instructions can help users navigate and utilize the online forum more effectively, improving accessibility.\n"Flexible and accessible intervention" (Stjernsward and Hansson, 2020): The interv

### separate out factors corresponding to three outcomes


In [11]:
def parse_factors(filename):
    # Initialize lists to store factors
    adherence_factors = []
    dropout_factors = []
    accessibility_factors = []

    # Variable to keep track of the current category
    current_category = None

    # Open and read the file line by line
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading and trailing whitespace
            if not line:  # Skip empty lines
                continue

            # Determine the current category
            if "The factors contributing to **Adherence** are" in line:
                current_category = "Adherence"
            elif "The factors contributing to **Dropout** are" in line:
                current_category = "Dropout"
            elif "The factors contributing to **Accessibility** are" in line:
                current_category = "Accessibility"
            
            # Separate factors into lists. 
            if current_category:
                # Count the factors that start with a dash
                if line.startswith("-"):
                    factor = line[1:].strip()  # Remove the dash and strip whitespace
                elif ":" in line:
                    # In case the factor is on the same line, after a colon
                    factor = line.split(":",1)[1].strip() or None
                    if factor==None:
                        continue
                if current_category == 'Adherence':
                    adherence_factors.append(factor)
                elif current_category == 'Dropout':
                    dropout_factors.append(factor)
                elif current_category == 'Accessibility':
                    accessibility_factors.append(factor)

    return adherence_factors, dropout_factors, accessibility_factors

# Use the function by providing the filename
# filename = os.path.join(work_dir, 'raw_factors/response_49_clean.txt')  # Replace with your actual file name
# adherence, dropout, accessibility = parse_factors(filename)

# print("Adherence Factors:", adherence)
# print("Dropout Factors:", dropout)
# print("Accessibility Factors:", accessibility)


In [12]:
# set directory

response_folder = 'work_dir/raw_factors/'
output_folder = 'work_dir/factors/'
output_folder_adherence = os.path.join(output_folder, 'adherence')
output_folder_accessibility = os.path.join(output_folder, 'accessibility')
output_folder_dropout = os.path.join(output_folder, 'dropout')
os.makedirs(output_folder_adherence, exist_ok=True)
os.makedirs(output_folder_accessibility, exist_ok=True)
os.makedirs(output_folder_dropout, exist_ok=True)

input_files = glob.glob(os.path.join(response_folder, 'response_cleaned', '*.txt'), recursive=True)


In [14]:

for input_file in input_files:
    adherence, dropout, accessibility = parse_factors(input_file)
    with open(os.path.join(output_folder_adherence, os.path.splitext(os.path.basename(input_file))[0] + '.txt'), 'w') as file1:
        for factor in adherence:
            file1.write(f"{factor}\n")
    with open(os.path.join(output_folder_accessibility, os.path.splitext(os.path.basename(input_file))[0] + '.txt'), 'w') as file1:
        for factor in accessibility:
            file1.write(f"{factor}\n")
    with open(os.path.join(output_folder_dropout, os.path.splitext(os.path.basename(input_file))[0] + '.txt'), 'w') as file1:
        for factor in dropout:
            file1.write(f"{factor}\n")
    


# # Write the factors to separate file
# response_folder = 'work_dir/raw_factors/'
# with open(os.path.join(response_folder,'adherence.txt'), 'w') as file1:
#     for factor in adherence:
#         file1.write(f"{factor}\n")

# with open(os.path.join(response_folder,'dropout.txt'), 'w') as file2:
#     for factor in dropout:
#         file2.write(f"{factor}\n")
        
# with open(os.path.join(response_folder,'accessibility.txt'), 'w') as file3:
#     for factor in accessibility:
#         file3.write(f"{factor}\n")

### Generate intermediate integrated factors

In [4]:
# set directory

response_folder = 'work_dir/raw_factors/'
output_folder = 'work_dir/factors/'
output_folder_adherence = os.path.join(output_folder, 'adherence')
output_folder_accessibility = os.path.join(output_folder, 'accessibility')
output_folder_dropout = os.path.join(output_folder, 'dropout')
os.makedirs(output_folder_adherence, exist_ok=True)
os.makedirs(output_folder_accessibility, exist_ok=True)
os.makedirs(output_folder_dropout, exist_ok=True)

input_files = glob.glob(os.path.join(response_folder, 'response_cleaned', '*.txt'), recursive=True)


In [ ]:
'''
Process all papers in one doc. 
'''
'''
    outcome_definition[0]: adherence
    outcome_definition[1]: dropout
    outcome_definition[2]: accessibility
'''
'''
    This block is for adherence. 
'''

loader = DirectoryLoader(path=output_folder_adherence, glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
documents = loader.load()

# Define chunking write-to directory
chunk_folder = os.path.join(output_folder,'chunks')
os.makedirs(chunk_folder, exist_ok=True)
# Chunk
chunk = chunking(chunk_folder,chunk_size=5000)


target_outcome = outcome_definition[0]
print(f'Consolidating factors in all papers for {target_outcome}')

chunks = chunk.make_chunks(documents)
print(f'number of chunks = {len(chunks)}')
outcome_filename = 'outcome_definition.json'
with open(os.path.join(path, outcome_filename)) as outcome_file:
    outcome_definition = json.load(outcome_file)

# generate_prompt = prompt(chunked_docs = documents, outcome_definition = outcome_definition)
# message_func = generate_prompt._message_find_factors_to_result
# results = generate_prompt.generate_response(message_func, include_chunk=True)
# generate_prompt.write_summaries_to_txt(results, 'response.txt')

generate_prompt = prompt(chunked_docs=chunks,outcome_definition=target_outcome,max_tokens=5000)
message_func = generate_prompt._message_consolidate_factors
results = generate_prompt.generate_response(message_func, include_chunk=False)
generate_prompt.write_summaries_to_txt(results, os.path.join(output_folder,'intermediate/adherence', 'adherence.txt'))

100%|██████████| 15/15 [00:00<00:00, 3445.11it/s]


Consolidating factors in all papers for {'phenomenon': 'Adherence', 'definition': 'Remaining engaged with and consistently following the intended course of a digital mindfulness-based intervention. This includes sustained participation, regular practice, completion of program activities, and factors that enable individuals to “stick with” or stay with the intervention or their mindfulness practice over time. '}
number of chunks = 33
processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3
processing chunk 4
processing chunk 5
processing chunk 6
processing chunk 7
processing chunk 8
processing chunk 9
processing chunk 10
processing chunk 11
processing chunk 12
processing chunk 13
processing chunk 14
processing chunk 15
processing chunk 16
processing chunk 17
processing chunk 18
processing chunk 19
processing chunk 20
processing chunk 21
processing chunk 22
processing chunk 23
processing chunk 24
processing chunk 25
processing chunk 26
processing chunk 27
processing ch

In [ ]:
'''
Process individual papers. 
'''
'''
    outcome_definition[0]: adherence
    outcome_definition[1]: dropout
    outcome_definition[2]: accessibility
'''
'''
    This block is for adherence. 
'''

loader = DirectoryLoader(path=output_folder_adherence, glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
documents = loader.load()

# Define chunking write-to directory
chunk_folder = os.path.join(output_folder,'chunks')
os.makedirs(chunk_folder, exist_ok=True)
# Chunk
chunk = chunking(chunk_folder,chunk_size=5000)

for i in range(len(documents)):
    # print(documents[i])
    # break
    target_outcome = outcome_definition[0]
    doc_index = re.search(r'_(\d+)_', os.path.splitext(os.path.basename(documents[i].metadata['source']))[0]).group(1)   # eg. take 118 from document's metadata: 'work_dir/factors/adherence/response_118_clean.txt'. 
    print(f'Consolidating factors in paper {doc_index} for {target_outcome}')
    
    chunks = chunk.make_chunks([documents[i]])
    print(f'number of chunks = {len(chunks)}')
    outcome_filename = 'outcome_definition.json'
    with open(os.path.join(path, outcome_filename)) as outcome_file:
        outcome_definition = json.load(outcome_file)

    # generate_prompt = prompt(chunked_docs = documents, outcome_definition = outcome_definition)
    # message_func = generate_prompt._message_find_factors_to_result
    # results = generate_prompt.generate_response(message_func, include_chunk=True)
    # generate_prompt.write_summaries_to_txt(results, 'response.txt')

    generate_prompt = prompt(chunked_docs=chunks,outcome_definition=target_outcome,max_tokens=5000)
    message_func = generate_prompt._message_consolidate_factors
    results = generate_prompt.generate_response(message_func, include_chunk=False)
    generate_prompt.write_summaries_to_txt(results, os.path.join(output_folder,'intermediate/adherence', f'{doc_index}adherence.txt'))

100%|██████████| 8/8 [00:00<00:00, 2646.46it/s]


Consolidating factors in paper 118 for {'phenomenon': 'Adherence', 'definition': 'Remaining engaged with and consistently following the intended course of a digital mindfulness-based intervention. This includes sustained participation, regular practice, completion of program activities, and factors that enable individuals to “stick with” or stay with the intervention or their mindfulness practice over time. '}
number of chunks = 9
processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3
processing chunk 4
processing chunk 5
processing chunk 6
processing chunk 7
processing chunk 8
Analyze results saved to text file. 
Consolidating factors in paper 49 for {'phenomenon': 'Adherence', 'definition': 'Remaining engaged with and consistently following the intended course of a digital mindfulness-based intervention. This includes sustained participation, regular practice, completion of program activities, and factors that enable individuals to “stick with” or stay with the i

In [15]:
chunks

[Document(metadata={'source': 'work_dir/factors/adherence/response_49_clean.txt'}, page_content='"Better support for the care recipient" (Atreya et al., 2018)\n"Desire to help with research" (Atreya et al., 2018)\n"Learn a new skill" (Atreya et al., 2018)\n"Help with sleep" (Atreya et al., 2018)\n"Promote relaxation" (Atreya et al., 2018)\n"Curiosity" (Atreya et al., 2018)\n"Assist to focus/train/organise thoughts" (Atreya et al., 2018)\n"Positive experience with meditation" (Atreya et al., 2018)\n"Assist to stay in the present" (Dragomanovich et al., 2021)\n"Help with sleep" (Dragomanovich et al., 2021)\n"Assist to focus/train/organise thoughts" (Dragomanovich et al., 2021)\n"Learn a new skill" (Dragomanovich et al., 2021)\n"Reduce anxiety" (Dragomanovich et al., 2021)\n"Flexible and accessible interventions" (Lange, 2020)\n"Improve coping skills" (Lange, 2020)\n"Opportunity for self-care" (Lange, 2020)\n"Positive experience with meditation" (Lange, 2020)\n"More mid-week practice remi

In [ ]:
## final integration of all papers
loader = DirectoryLoader(os.path.join(output_folder, 'intermediate','adherence','together'), show_progress=True, loader_cls=TextLoader)
documents = loader.load()

generate_prompt = prompt(chunked_docs=documents,outcome_definition=outcome_definition[0],max_tokens=16384)
message_func = generate_prompt._message_final_factors
results = generate_prompt.generate_response(message_func, include_chunk=False)
generate_prompt.write_summaries_to_txt(results, os.path.join(output_folder,'final','adherence.txt'))


100%|██████████| 1/1 [00:00<00:00, 1076.57it/s]


In [ ]:
## final integration of individual paper. 
loader = DirectoryLoader(os.path.join(output_folder, 'intermediate','adherence'), show_progress=True, loader_cls=TextLoader)
documents = loader.load()

for i in range(len(documents)):
    doc_name = os.path.splitext(os.path.basename(documents[i].metadata['source']))[0]
    generate_prompt = prompt(chunked_docs=[documents[i]],outcome_definition=outcome_definition[0],max_tokens=16384)
    message_func = generate_prompt._message_final_factors
    results = generate_prompt.generate_response(message_func, include_chunk=False)
    generate_prompt.write_summaries_to_txt(results, os.path.join(output_folder,'final','adherence',f'{doc_name}.txt'))


100%|██████████| 9/9 [00:00<00:00, 1896.83it/s]


processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 
processing chunk 0
Analyze results saved to text file. 


In [20]:
results

['1. **Structured and Manageable Practice**\n   - Description: Establishing a specific, manageable daily meditation routine, such as a 10-minute session, and using structured courses like "Take Ten" enhances adherence by providing clear guidelines and convenience.\n   - Citations: 3 (Baydoun et al., 2021; unspecified studies with 51% adherence; unspecified studies with good adherence rates for structured courses)\n   - Frequency: high frequency\n   - Study_id: {\'work_dir/factors/adherence/response_623_clean.txt\'}\n\n2. **Positive Outcomes and Feedback**\n   - Description: Experiencing positive outcomes, such as reductions in anxiety, is linked to longer and more frequent use of mindfulness applications, encouraging continued engagement.\n   - Citations: 3 (unspecified studies showing greater reductions in anxiety with longer use; unspecified studies linking usage time to symptom decrease; unspecified studies indicating improved anxiety symptoms with 10+ sessions)\n   - Frequency: hig

In [17]:
os.path.join(output_folder, 'adherence','final',doc_index)

'work_dir/factors/adherence/final/642'

In [11]:
generate_prompt.write_summaries_to_txt(results, os.path.join(output_folder, 'adherence_final.txt'))

Analyze results saved to text file. 
